<a href="https://colab.research.google.com/github/jani0x0/Coursera_Capstone/blob/main/Final_project_Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Exploration around popular spots**

In [1]:
# Get list of districts and filter Cusco and Lima region
from bs4 import BeautifulSoup
import requests
import pandas as pd

page = requests.get("https://es.wikipedia.org/wiki/Anexo:Distritos_del_Per%C3%BA")
soup = BeautifulSoup(page.content, 'html.parser')

table_contents=[]
table=soup.find('table')
table_rows = table.find_all('tr')

for tr in table_rows:
    td = tr.find_all('td')
    coltext = [col.text for col in td]
    if coltext!=[]:
        table_contents.append(coltext)

peru_districts=pd.DataFrame(table_contents, columns=["id", "Region", "Province", "District", "Population"])
peru_districts.drop(['id', 'Population'], axis=1, inplace=True)
cusco_data=peru_districts.loc[peru_districts['Region'].str.contains("Cusco")]
lima_data=peru_districts.loc[peru_districts['Region'].str.contains("Lima")]
print(cusco_data.head())
print(lima_data.head())

    Region Province       District
696  Cusco    Cusco          Cusco
697  Cusco    Cusco         Ccorca
698  Cusco    Cusco          Poroy
699  Cusco    Cusco   San Jerónimo
700  Cusco    Cusco  San Sebastián
     Region Province  District
1278   Lima     Lima      Lima
1279   Lima     Lima     Ancón
1280   Lima     Lima       Ate
1281   Lima     Lima  Barranco
1282   Lima     Lima     Breña


In [2]:
# Getting coordinates

CLIENT_ID_POS = '78ee34f6037969c51976101a2ca58d41'

# WORKING WITH CUSCO DATA:

for index, row in cusco_data.iterrows():
    latitude = None
    longitude = None
    query_str=row['District']
    url_pos = 'http://api.positionstack.com/v1/forward?access_key={}&query={}&region={}'.format(CLIENT_ID_POS, query_str,'Cusco')
    results = requests.get(url_pos).json()
    if results['data'] is not None:
        for elem in results['data']:
            if (elem['type']=='locality') and (elem['county']==row['Province']):
                latitude = elem['latitude']
                longitude = elem['longitude']
                break
        cusco_data.loc[index, 'Latitude'] = latitude
        cusco_data.loc[index, 'Longitude'] = longitude

cusco_data

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Region,Province,District,Latitude,Longitude
696,Cusco,Cusco,Cusco,-13.551969,-72.069168
697,Cusco,Cusco,Ccorca,-13.585000,-72.059720
698,Cusco,Cusco,Poroy,-13.495000,-72.043330
699,Cusco,Cusco,San Jerónimo,-13.545940,-71.884780
700,Cusco,Cusco,San Sebastián,-13.531380,-71.931430
...,...,...,...,...,...
803,Cusco,Urubamba,Huayllabamba,-13.338060,-72.065280
804,Cusco,Urubamba,Machupicchu,-13.158330,-72.531390
805,Cusco,Urubamba,Maras,-13.335000,-72.156670
806,Cusco,Urubamba,Ollantaytambo,-13.265008,-72.267208


In [3]:
# REMOVING ROWS THAT COULDNT GET COORDINATES
import numpy as np

cusco_tmp=cusco_data.replace(r'^\s*$', np.nan, regex=True)
cusco_df=cusco_tmp.dropna(axis=0, inplace=False)
cusco_df


,Region,Province,District,Latitude,Longitude
696,Cusco,Cusco,Cusco,-13.551969,-72.069168
697,Cusco,Cusco,Ccorca,-13.585000,-72.059720
698,Cusco,Cusco,Poroy,-13.495000,-72.043330
699,Cusco,Cusco,San Jerónimo,-13.545940,-71.884780
700,Cusco,Cusco,San Sebastián,-13.531380,-71.931430
...,...,...,...,...,...
803,Cusco,Urubamba,Huayllabamba,-13.338060,-72.065280
804,Cusco,Urubamba,Machupicchu,-13.158330,-72.531390
805,Cusco,Urubamba,Maras,-13.335000,-72.156670
806,Cusco,Urubamba,Ollantaytambo,-13.265008,-72.267208


In [4]:
# WORKING WITH LIMA DATA:

for index, row in lima_data.iterrows():
    latitude = None
    longitude = None
    query_str=row['District']
    url_pos = 'http://api.positionstack.com/v1/forward?access_key={}&query={}&region={}'.format(CLIENT_ID_POS, query_str, 'Lima')
    results = requests.get(url_pos).json()
    if results['data'] is not None:
        for elem in results['data']:
            if (elem['type']=='locality') and (elem['county']==row['Province']):
                latitude = elem['latitude']
                longitude = elem['longitude']
                break    
        lima_data.loc[index, 'Latitude'] = latitude
        lima_data.loc[index, 'Longitude'] = longitude

lima_data

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Region,Province,District,Latitude,Longitude
1278,Lima,Lima,Lima,-12.049141,-77.021217
1279,Lima,Lima,Ancón,-11.774270,-77.170060
1280,Lima,Lima,Ate,-12.050000,-76.966670
1281,Lima,Lima,Barranco,NaN,NaN
1282,Lima,Lima,Breña,NaN,NaN
...,...,...,...,...,...
1444,Lima,Yauyos,Tauripampa,-12.583330,-76.116670
1445,Lima,Yauyos,Tomas,-12.238610,-75.745280
1446,Lima,Yauyos,Tupe,-12.740830,-75.808330
1447,Lima,Yauyos,Viñac,-12.931110,-75.778610


In [5]:
# REMOVING ROWS WITHOUT COORDINATES

lima_tmp=lima_data.replace(r'^\s*$', np.nan, regex=True)
lima_df=lima_tmp.dropna(axis=0, inplace=False)

lima_df

,Region,Province,District,Latitude,Longitude
1278,Lima,Lima,Lima,-12.049141,-77.021217
1279,Lima,Lima,Ancón,-11.774270,-77.170060
1280,Lima,Lima,Ate,-12.050000,-76.966670
1283,Lima,Lima,Carabayllo,-11.853330,-77.037780
1284,Lima,Lima,Chaclacayo,-12.068195,-76.861999
...,...,...,...,...,...
1444,Lima,Yauyos,Tauripampa,-12.583330,-76.116670
1445,Lima,Yauyos,Tomas,-12.238610,-75.745280
1446,Lima,Yauyos,Tupe,-12.740830,-75.808330
1447,Lima,Yauyos,Viñac,-12.931110,-75.778610



Now, the districts where the most popular sites are located will be explored.

First, Cusco region will be explored. The districts to be considered are where Machu Picchu, Cusco city, Aguas Calientes, Ollantaytambo, Valle Sagrado and Pisac are:

| District | Province | Region |
| - | - | - |
| Machupicchu | Urubamba | Cusco |
| Ollantaytambo | Urubamba | Cusco |
| Yucay | Urubamba | Cusco |
| Cusco | Cusco | Cusco |
| Písac | Calca | Cusco |


In [6]:
# Defining Foursquare API input details

#CLIENT_ID = 'UT20ZRXHFJHWEEPXAYDIXVXFFSKRASXV0PX3RDRUYSLKTAP0' # your Foursquare ID
#CLIENT_SECRET = 'DZY2XAXW1XR2GG12WL5IJ1QLPAKCD1WHNVZKE0YKSID2R15P' # your Foursquare Secret
#ACCESS_TOKEN = 'PUF1SI0RET5ZLDYCHKFTASEMER4TSWEIGJ3U30PDHLIYPKJK'
CLIENT_ID = 'GRSUXZUAI0RVX5F2JKENWPSK2JAFYLKJKVGRX2Z4GOGXIHOI' # your Foursquare ID
CLIENT_SECRET = 'LUXZ3TNF04ZUAOUBA1OF0JQ0R3XNGBA202YDZZGCSIFW1ATQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

exp_dist=['Machupicchu', 'Ollantaytambo', 'Yucay', 'Cusco', 'Písac']
cusco_exp=cusco_df[cusco_df.District.isin(exp_dist)]

# create map of selected districts
import folium
map_cuscodist = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, District, Province in zip(cusco_exp['Latitude'], cusco_exp['Longitude'], cusco_exp['District'], cusco_exp['Province']):
    label = '{}, {}'.format(District, Province)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cuscodist)  
    
map_cuscodist


In [7]:
# Function to get venues in each district
def getNearbyVenues(names, latitudes, longitudes, radius=800):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng,
            v['venue']['id'],
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude',
                  'Venue ID', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
# Getting venues and categories in selected districts in Cusco region

cuscodist_venues_withid = getNearbyVenues(names=cusco_exp['District'],
                                   latitudes=cusco_exp['Latitude'],
                                   longitudes=cusco_exp['Longitude']
                                  )
cuscodist_venues = cuscodist_venues_withid.drop(columns=['Venue ID'], inplace=False)
print(cuscodist_venues.shape)
cuscodist_venues.head()

(52, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Písac,-13.4225,-71.84667,Cafe Mullu,-13.421354,-71.850349,Café
1,Písac,-13.4225,-71.84667,Plaza de Pisac,-13.421107,-71.850479,Plaza
2,Písac,-13.4225,-71.84667,Horno Pumachayoc,-13.420645,-71.847854,Bakery
3,Písac,-13.4225,-71.84667,Mercado Abierto de Pisac,-13.421070,-71.850522,Market
4,Písac,-13.4225,-71.84667,Ulrike's Cafe,-13.421312,-71.851043,Café


In [9]:
# Venues grouped by District
cuscodist_venues.groupby('District').count()

,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
District,,,,,,
Machupicchu,28,28,28,28,28,28
Ollantaytambo,10,10,10,10,10,10
Písac,9,9,9,9,9,9
Yucay,5,5,5,5,5,5


In [10]:
print('There are {} uniques categories.'.format(len(cuscodist_venues['Venue Category'].unique())))

There are 19 uniques categories.


In [11]:
# Analyzing each selected district

# one hot encoding
cuscodist_onehot = pd.get_dummies(cuscodist_venues[['Venue Category']], prefix="", prefix_sep="")

# add district column back to dataframe
cuscodist_onehot['District'] = cuscodist_venues['District'] 

# move district column to the first column
fixed_columns = [cuscodist_onehot.columns[-1]] + list(cuscodist_onehot.columns[:-1])
cuscodist_onehot = cuscodist_onehot[fixed_columns]
cuscodist_grouped = cuscodist_onehot.groupby('District').mean().reset_index()

# Getting top 5 venues' categories in each district

num_top_venues = 5

for hood in cuscodist_grouped['District']:
    print("----"+hood+"----")
    temp = cuscodist_grouped[cuscodist_grouped['District'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Machupicchu----
                 venue  freq
0                Hotel  0.29
1                Trail  0.11
2               Hostel  0.11
3  Peruvian Restaurant  0.11
4      Bed & Breakfast  0.07


----Ollantaytambo----
                 venue  freq
0                Hotel   0.4
1        Train Station   0.2
2                 Café   0.2
3  Peruvian Restaurant   0.1
4       Breakfast Spot   0.1


----Písac----
        venue  freq
0        Café  0.22
1      Bakery  0.11
2       Hotel  0.11
3  Restaurant  0.11
4       Plaza  0.11


----Yucay----
                 venue  freq
0  Peruvian Restaurant   0.4
1                 Café   0.2
2                Hotel   0.2
3          Pizza Place   0.2
4               Bakery   0.0




In [12]:
# Function to write top venues in Dataframe

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [13]:
# Function to get 3 popular sites from a category in a district:
CLIENT_ID = 'GRSUXZUAI0RVX5F2JKENWPSK2JAFYLKJKVGRX2Z4GOGXIHOI' # your Foursquare ID
CLIENT_SECRET = 'LUXZ3TNF04ZUAOUBA1OF0JQ0R3XNGBA202YDZZGCSIFW1ATQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

def return_popvenues_dist (ids):
    venues_list=[]
    for id in ids:
      url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(id, CLIENT_ID, CLIENT_SECRET, VERSION)
      result = requests.get(url).json()
      print(result)  

In [14]:
# SHOWING DATAFRAME WITH TOP 10 VENUES CATEGORIES IN EACH DISTRICT

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
cuscodist_venues_sorted = pd.DataFrame(columns=columns)
cuscodist_venues_sorted['District'] = cuscodist_grouped['District']

for ind in np.arange(cuscodist_grouped.shape[0]):
    cuscodist_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cuscodist_grouped.iloc[ind, :], num_top_venues)

cuscodist_venues_sorted

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Machupicchu,Hotel,Trail,Peruvian Restaurant,Hostel,Bed & Breakfast,Restaurant,Zoo,Italian Restaurant,Buffet,Food
1,Ollantaytambo,Hotel,Café,Train Station,Breakfast Spot,Peruvian Restaurant,Zoo,Bed & Breakfast,Buffet,Food,Hostel
2,Písac,Café,Latin American Restaurant,Peruvian Restaurant,Hotel,Market,Bakery,Plaza,Restaurant,Pizza Place,Train Station
3,Yucay,Peruvian Restaurant,Hotel,Café,Pizza Place,Zoo,Bed & Breakfast,Breakfast Spot,Buffet,Food,Hostel


In [ ]:
# Find top 3 of venues per category and district from cusco_venues_withid dataframe

for dist, firstve in zip(cuscodist_venues_sorted['District'], cuscodist_venues_sorted['1st Most Common Venue']):
    tmpdf=cuscodist_venues_withid[cuscodist_venues_withid.District.isin([dist])]
    tmpdf1=tmpdf.loc[tmpdf['Venue Category'] == firstve]
    tmpdf1=tmpdf1['Venue ID']
    return_popvenues_dist(tmpdf1)

{'meta': {'code': 200, 'requestId': '6104113d8d13272ea3fc12b8'}, 'response': {'venue': {'id': '4da7e2770cb65e9aae5c9ed8', 'name': 'Sumaq Machu Picchu Hotel', 'contact': {'phone': '+5184211059', 'formattedPhone': '+51 84 211059'}, 'location': {'address': 'Av.Hermanos Ayar Mz 1 Lote 3', 'lat': -13.154715364974253, 'lng': -72.52819649107718, 'labeledLatLngs': [{'label': 'display', 'lat': -13.154715364974253, 'lng': -72.52819649107718}], 'cc': 'PE', 'city': 'Machu Picchu', 'state': 'Cusco', 'country': 'Perú', 'formattedAddress': ['Av.Hermanos Ayar Mz 1 Lote 3', 'Machu Picchu', 'Perú']}, 'canonicalUrl': 'https://foursquare.com/v/sumaq-machu-picchu-hotel/4da7e2770cb65e9aae5c9ed8', 'categories': [{'id': '4bf58dd8d48988d1fa931735', 'name': 'Hotel', 'pluralName': 'Hotels', 'shortName': 'Hotel', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_', 'suffix': '.png'}, 'primary': True}], 'verified': True, 'stats': {'tipCount': 33}, 'url': 'http://www.machupicchuhotels-sumaq.co

Then Lima region will be explored. The districts to be considered are:

| District | Province | Region |
| - | - | - |
| Lima | Lima | Lima |
| Miraflores | Lima | Lima |
| San Isidro | Lima | Lima |
| Barranco | Lima | Lima |

However, the coordinates for Barranco couldn't be retrieved from PositionStack API, so only 3 districts will be considered.

In [15]:
# create map of selected districts
exp_limadist=['Lima', 'Miraflores', 'San Isidro']
lima_temp=lima_df[lima_df.District.isin(exp_limadist)]
lima_exp=lima_temp[lima_temp.Province.isin(['Lima'])]

map_limadist = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, District, Province in zip(lima_exp['Latitude'], lima_exp['Longitude'], lima_exp['District'], lima_exp['Province']):
    label = '{}, {}'.format(District, Province)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_limadist)  
    
map_limadist

In [16]:
# Getting venues and categories in selected districts in Lima region

limadist_venues_withid = getNearbyVenues(names=lima_exp['District'],
                                   latitudes=lima_exp['Latitude'],
                                   longitudes=lima_exp['Longitude']
                                  )
limadist_venues = limadist_venues_withid.drop(columns=['Venue ID'], inplace=False)
print(limadist_venues.shape)
limadist_venues.head()

(198, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lima,-12.049141,-77.021217,Restaurante La Buena Muerte,-12.048123,-77.022988,Seafood Restaurant
1,Lima,-12.049141,-77.021217,Chifa Kong,-12.051862,-77.024901,Chinese Restaurant
2,Lima,-12.049141,-77.021217,Escuela Nacional Superior Autónoma de Bellas A...,-12.046799,-77.023974,College Academic Building
3,Lima,-12.049141,-77.021217,Wa Lok,-12.052227,-77.025085,Chinese Restaurant
4,Lima,-12.049141,-77.021217,Museo de la Moneda,-12.049476,-77.023991,History Museum


In [17]:
# Venues grouped by District
limadist_venues.groupby('District').count()

,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
District,,,,,,
Lima,23,23,23,23,23,23
Miraflores,100,100,100,100,100,100
San Isidro,75,75,75,75,75,75


In [18]:
print('There are {} uniques categories.'.format(len(limadist_venues['Venue Category'].unique())))

There are 76 uniques categories.


In [19]:
# Analyzing each selected district

# one hot encoding
limadist_onehot = pd.get_dummies(limadist_venues[['Venue Category']], prefix="", prefix_sep="")

# add district column back to dataframe
limadist_onehot['District'] = limadist_venues['District'] 

# move district column to the first column
fixed_columns = [limadist_onehot.columns[-1]] + list(limadist_onehot.columns[:-1])
limadist_onehot = limadist_onehot[fixed_columns]
limadist_grouped = limadist_onehot.groupby('District').mean().reset_index()

# Getting top 5 venues' categories in each district

num_top_venues = 5

for hood in limadist_grouped['District']:
    print("----"+hood+"----")
    temp = limadist_grouped[limadist_grouped['District'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Lima----
                venue  freq
0  Chinese Restaurant  0.26
1  Seafood Restaurant  0.13
2              Bakery  0.09
3      History Museum  0.04
4                 Spa  0.04


----Miraflores----
                venue  freq
0                Park  0.08
1                Café  0.06
2  Seafood Restaurant  0.06
3         Coffee Shop  0.05
4          Restaurant  0.05


----San Isidro----
                 venue  freq
0                Hotel  0.12
1  Peruvian Restaurant  0.08
2          Coffee Shop  0.07
3           Restaurant  0.05
4           Steakhouse  0.04




In [20]:
# SHOWING DATAFRAME WITH TOP 10 VENUES CATEGORIES IN EACH DISTRICT

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
limadist_venues_sorted = pd.DataFrame(columns=columns)
limadist_venues_sorted['District'] = limadist_grouped['District']

for ind in np.arange(limadist_grouped.shape[0]):
    limadist_venues_sorted.iloc[ind, 1:] = return_most_common_venues(limadist_grouped.iloc[ind, :], num_top_venues)

limadist_venues_sorted

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Lima,Chinese Restaurant,Seafood Restaurant,Bakery,Arts & Crafts Store,College Academic Building,Market,Park,Dim Sum Restaurant,Restaurant,Asian Restaurant
1,Miraflores,Park,Seafood Restaurant,Café,Coffee Shop,Restaurant,Italian Restaurant,Peruvian Restaurant,Chinese Restaurant,Hostel,Bed & Breakfast
2,San Isidro,Hotel,Peruvian Restaurant,Coffee Shop,Restaurant,Italian Restaurant,Steakhouse,Bistro,Hotel Bar,Park,Pizza Place


In [21]:
# Find top 3 of venues per category and district from lima_venues_withid dataframe
for dist, firstve in zip(limadist_venues_sorted['District'], limadist_venues_sorted['1st Most Common Venue']):
    tmpdf=limadist_venues_withid[limadist_venues_withid.District.isin([dist])]
    tmpdf1=tmpdf.loc[tmpdf['Venue Category'] == firstve]
    tmpdf1=tmpdf1['Venue ID']
    return_popvenues_dist(tmpdf1)

{'meta': {'code': 200, 'requestId': '6105039184bad413ea8879aa'}, 'response': {'venue': {'id': '59246e9bb6eedb5ddc3a32d4', 'name': 'Chifa Kong', 'contact': {}, 'location': {'address': 'Paruro 836', 'lat': -12.051862, 'lng': -77.024901, 'labeledLatLngs': [{'label': 'display', 'lat': -12.051862, 'lng': -77.024901}], 'cc': 'PE', 'city': 'Lima', 'state': 'Lima', 'country': 'Perú', 'formattedAddress': ['Paruro 836', 'Lima', 'Perú']}, 'canonicalUrl': 'https://foursquare.com/v/chifa-kong/59246e9bb6eedb5ddc3a32d4', 'categories': [{'id': '4bf58dd8d48988d145941735', 'name': 'Chinese Restaurant', 'pluralName': 'Chinese Restaurants', 'shortName': 'Chinese', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_', 'suffix': '.png'}, 'primary': True}], 'verified': False, 'stats': {'tipCount': 15}, 'price': {'tier': 1, 'message': 'Cheap', 'currency': '$'}, 'likes': {'count': 15, 'groups': [{'type': 'others', 'count': 15, 'items': []}], 'summary': '15 Likes'}, 'dislike': False, 'ok': Fa

# **Segmentation**

Following the second purpose of this project, we will segments spots located in each region.
First, we start with Cusco.

In [23]:
# Getting all venues in Cusco region

cusco_venues_withid = getNearbyVenues(names=cusco_df['District'],
                                   latitudes=cusco_df['Latitude'],
                                   longitudes=cusco_df['Longitude'])
cusco_venues = cusco_venues_withid.drop(columns=['Venue ID'], inplace=False)
print(cusco_venues.shape)
cusco_venues.head()

(208, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ccorca,-13.585,-72.05972,Plaza Ccorca,-13.582261,-72.060280,Plaza
1,Poroy,-13.495,-72.04333,PeruRail - Estación Poroy | Poroy Station (Est...,-13.494642,-72.042468,Train Station
2,Poroy,-13.495,-72.04333,La Olla Peruana,-13.493507,-72.042513,Peruvian Restaurant
3,Poroy,-13.495,-72.04333,Arco de Ticatica,-13.494541,-72.042780,Historic Site
4,Poroy,-13.495,-72.04333,Chicharroneria Santa Barbara,-13.493830,-72.045506,BBQ Joint


In [24]:
print('There are {} uniques categories in Cusco.'.format(len(cusco_venues['Venue Category'].unique())))

There are 52 uniques categories in Cusco.


In [25]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# one hot encoding
cusco_onehot = pd.get_dummies(cusco_venues[['Venue Category']], prefix="", prefix_sep="")

# add district column back to dataframe
cusco_onehot['District'] = cusco_venues['District'] 

# move district column to the first column
fixed_columns = [cusco_onehot.columns[-1]] + list(cusco_onehot.columns[:-1])
cusco_onehot = cusco_onehot[fixed_columns]
cusco_grouped = cusco_onehot.groupby('District').mean().reset_index()

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
cusco_venues_sorted = pd.DataFrame(columns=columns)
cusco_venues_sorted['District'] = cusco_grouped['District']

for ind in np.arange(cusco_grouped.shape[0]):
    cusco_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cusco_grouped.iloc[ind, :], num_top_venues)


cusco_grouped_clustering = cusco_grouped.drop('District', 1)

In [26]:
# set number of clusters
kclusters_cusco = 7

# run k-means clustering
kmeans_cusco = KMeans(n_clusters=kclusters_cusco, random_state=0).fit(cusco_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans_cusco.labels_[0:10] 

array([6, 5, 2, 0, 4, 0, 2, 0, 0, 0], dtype=int32)

In [27]:
# add clustering labels
cusco_venues_sorted.insert(0, 'Cluster Labels', kmeans_cusco.labels_)

cusco_merged = cusco_df

cusco_merged = cusco_merged.join(cusco_venues_sorted.set_index('District'), on='District')

cusco_merged.head()

,Region,Province,District,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
696,Cusco,Cusco,Cusco,-13.551969,-72.069168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
697,Cusco,Cusco,Ccorca,-13.585000,-72.059720,2.0,Plaza,Zoo,History Museum,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Food,Flea Market,Field
698,Cusco,Cusco,Poroy,-13.495000,-72.043330,0.0,Peruvian Restaurant,Historic Site,Train Station,BBQ Joint,Farm,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Food
699,Cusco,Cusco,San Jerónimo,-13.545940,-71.884780,0.0,Hotel,Latin American Restaurant,Flea Market,Plaza,Zoo,Farm,General Entertainment,Gas Station,Fried Chicken Joint,Food
700,Cusco,Cusco,San Sebastián,-13.531380,-71.931430,0.0,Grocery Store,Hotel,Supermarket,Plaza,Clothing Store,General Entertainment,Gas Station,Fried Chicken Joint,Food,Flea Market


In [28]:
cusco_merged.dropna(subset = ["Cluster Labels"], inplace=True)
cusco_merged.head()

,Region,Province,District,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
697,Cusco,Cusco,Ccorca,-13.58500,-72.05972,2.0,Plaza,Zoo,History Museum,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Food,Flea Market,Field
698,Cusco,Cusco,Poroy,-13.49500,-72.04333,0.0,Peruvian Restaurant,Historic Site,Train Station,BBQ Joint,Farm,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Food
699,Cusco,Cusco,San Jerónimo,-13.54594,-71.88478,0.0,Hotel,Latin American Restaurant,Flea Market,Plaza,Zoo,Farm,General Entertainment,Gas Station,Fried Chicken Joint,Food
700,Cusco,Cusco,San Sebastián,-13.53138,-71.93143,0.0,Grocery Store,Hotel,Supermarket,Plaza,Clothing Store,General Entertainment,Gas Station,Fried Chicken Joint,Food,Flea Market
701,Cusco,Cusco,Santiago,-13.52728,-71.98430,0.0,Hotel,Plaza,Hostel,Bus Station,Farmers Market,Pizza Place,Farm,General Entertainment,Gas Station,Fried Chicken Joint


In [29]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_cuscoclus = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters_cusco)
ys = [i + x + (i*x)**2 for i in range(kclusters_cusco)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cusco_merged['Latitude'], cusco_merged['Longitude'], cusco_merged['District'], cusco_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_cuscoclus)
       
map_cuscoclus

In [30]:
#CLUSTER 1 - Cusco
cusco_merged.loc[cusco_merged['Cluster Labels'] == 0, cusco_merged.columns[[2] + list(range(5, cusco_merged.shape[1]))]]

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
698,Poroy,0.0,Peruvian Restaurant,Historic Site,Train Station,BBQ Joint,Farm,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Food
699,San Jerónimo,0.0,Hotel,Latin American Restaurant,Flea Market,Plaza,Zoo,Farm,General Entertainment,Gas Station,Fried Chicken Joint,Food
700,San Sebastián,0.0,Grocery Store,Hotel,Supermarket,Plaza,Clothing Store,General Entertainment,Gas Station,Fried Chicken Joint,Food,Flea Market
701,Santiago,0.0,Hotel,Plaza,Hostel,Bus Station,Farmers Market,Pizza Place,Farm,General Entertainment,Gas Station,Fried Chicken Joint
703,Wánchaq,0.0,Hotel,Bed & Breakfast,Seafood Restaurant,Bakery,Pizza Place,Breakfast Spot,Shopping Mall,Plaza,Soccer Stadium,South American Restaurant
713,Cachimayo,0.0,General Entertainment,Bike Trail,Zoo,Farm,Historic Site,Grocery Store,Gas Station,Fried Chicken Joint,Food,Flea Market
717,Mollepata,0.0,Bed & Breakfast,Restaurant,Peruvian Restaurant,Farmers Market,Zoo,Farm,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
721,Coya,0.0,Trail,Peruvian Restaurant,Plaza,Zoo,Clothing Store,General Entertainment,Gas Station,Fried Chicken Joint,Food,Flea Market
722,Lamay,0.0,Resort,Latin American Restaurant,Fast Food Restaurant,Clothing Store,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Food,Flea Market
724,Písac,0.0,Café,Peruvian Restaurant,Hotel,Bakery,Latin American Restaurant,Market,Restaurant,Plaza,Zoo,Gas Station


In [31]:
#CLUSTER 2 - Cusco
cusco_merged.loc[cusco_merged['Cluster Labels'] == 1, cusco_merged.columns[[2] + list(range(5, cusco_merged.shape[1]))]]

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
803,Huayllabamba,1.0,Trail,Zoo,Clothing Store,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Food,Flea Market,Field


In [32]:
#CLUSTER 3 - Cusco
cusco_merged.loc[cusco_merged['Cluster Labels'] == 2, cusco_merged.columns[[2] + list(range(5, cusco_merged.shape[1]))]]

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
697,Ccorca,2.0,Plaza,Zoo,History Museum,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Food,Flea Market,Field
706,Acos,2.0,Plaza,Zoo,History Museum,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Food,Flea Market,Field
718,Pucyura,2.0,Plaza,Zoo,History Museum,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Food,Flea Market,Field
774,Paruro,2.0,Shopping Mall,Plaza,Zoo,Clothing Store,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Food,Flea Market


In [33]:
#CLUSTER 4 - Cusco
cusco_merged.loc[cusco_merged['Cluster Labels'] == 3, cusco_merged.columns[[2] + list(range(5, cusco_merged.shape[1]))]]

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
716,Limatambo,3.0,History Museum,Vegetarian / Vegan Restaurant,Historic Site,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Food,Flea Market,Field
742,San Pedro,3.0,History Museum,Historic Site,Vegetarian / Vegan Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Food,Flea Market,Field
743,Tinta,3.0,History Museum,Vegetarian / Vegan Restaurant,Historic Site,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Food,Flea Market,Field


In [34]:
#CLUSTER 5 - Cusco
cusco_merged.loc[cusco_merged['Cluster Labels'] == 4, cusco_merged.columns[[2] + list(range(5, cusco_merged.shape[1]))]]

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
711,Anta,4.0,Soccer Field,Zoo,Clothing Store,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Food,Flea Market,Field


In [35]:
#CLUSTER 6 - Cusco
cusco_merged.loc[cusco_merged['Cluster Labels'] == 5, cusco_merged.columns[[2] + list(range(5, cusco_merged.shape[1]))]]

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
705,Acopía,5.0,Lake,Zoo,History Museum,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Food,Flea Market,Field
796,Lucre,5.0,Lake,Zoo,History Museum,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Food,Flea Market,Field


In [36]:
#CLUSTER 7 - Cusco
cusco_merged.loc[cusco_merged['Cluster Labels'] == 6, cusco_merged.columns[[2] + list(range(5, cusco_merged.shape[1]))]]

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
704,Acomayo,6.0,Park,Zoo,History Museum,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Food,Flea Market,Field
766,Kimbiri,6.0,Park,Restaurant,Zoo,Clothing Store,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Food,Flea Market
782,Yaurisque,6.0,Park,Restaurant,Plaza,Zoo,Clothing Store,General Entertainment,Gas Station,Fried Chicken Joint,Food,Flea Market


Now, we segment spots in Lima region:

In [37]:
# Getting venues and categories in selected districts in Lima region

lima_venues_withid = getNearbyVenues(names=lima_df['District'],
                                   latitudes=lima_df['Latitude'],
                                   longitudes=lima_df['Longitude']
                                  )
lima_venues = lima_venues_withid.drop(columns=['Venue ID'], inplace=False)
print(lima_venues.shape)
lima_venues.head()

(830, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lima,-12.049141,-77.021217,Restaurante La Buena Muerte,-12.048123,-77.022988,Seafood Restaurant
1,Lima,-12.049141,-77.021217,Chifa Kong,-12.051862,-77.024901,Chinese Restaurant
2,Lima,-12.049141,-77.021217,Escuela Nacional Superior Autónoma de Bellas A...,-12.046799,-77.023974,College Academic Building
3,Lima,-12.049141,-77.021217,Wa Lok,-12.052227,-77.025085,Chinese Restaurant
4,Lima,-12.049141,-77.021217,Museo de la Moneda,-12.049476,-77.023991,History Museum


In [38]:
print('There are {} uniques categories in Lima.'.format(len(lima_venues['Venue Category'].unique())))

There are 161 uniques categories in Lima.


In [39]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# one hot encoding
lima_onehot = pd.get_dummies(lima_venues[['Venue Category']], prefix="", prefix_sep="")

# add district column back to dataframe
lima_onehot['District'] = lima_venues['District'] 

# move district column to the first column
fixed_columns = [lima_onehot.columns[-1]] + list(lima_onehot.columns[:-1])
lima_onehot = lima_onehot[fixed_columns]
lima_grouped = lima_onehot.groupby('District').mean().reset_index()

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
lima_venues_sorted = pd.DataFrame(columns=columns)
lima_venues_sorted['District'] = lima_grouped['District']

for ind in np.arange(lima_grouped.shape[0]):
    lima_venues_sorted.iloc[ind, 1:] = return_most_common_venues(lima_grouped.iloc[ind, :], num_top_venues)


lima_grouped_clustering = lima_grouped.drop('District', 1)

# set number of clusters
kclusters_lima = 5

# run k-means clustering
kmeans_lima = KMeans(n_clusters=kclusters_lima, random_state=0).fit(lima_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans_lima.labels_[0:10]

array([1, 0, 1, 1, 1, 1, 1, 2, 2, 0], dtype=int32)

In [40]:
# add clustering labels
lima_venues_sorted.insert(0, 'Cluster Labels', kmeans_lima.labels_)

lima_merged = lima_df

lima_merged = lima_merged.join(lima_venues_sorted.set_index('District'), on='District')

lima_merged.head()

,Region,Province,District,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1278,Lima,Lima,Lima,-12.049141,-77.021217,1.0,Chinese Restaurant,Seafood Restaurant,Bakery,College Academic Building,Dim Sum Restaurant,Miscellaneous Shop,Spa,Furniture / Home Store,Market,Bookstore
1279,Lima,Lima,Ancón,-11.774270,-77.170060,1.0,Seafood Restaurant,Beach,Dessert Shop,Farm,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant
1280,Lima,Lima,Ate,-12.050000,-76.966670,1.0,Park,Fast Food Restaurant,Dessert Shop,Fried Chicken Joint,Soccer Field,Soccer Stadium,Department Store,Seafood Restaurant,Burger Joint,Chinese Restaurant
1283,Lima,Lima,Carabayllo,-11.853330,-77.037780,0.0,Latin American Restaurant,Peruvian Restaurant,Falafel Restaurant,Food,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
1284,Lima,Lima,Chaclacayo,-12.068195,-76.861999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
lima_merged.dropna(subset = ["Cluster Labels"], inplace=True)
lima_merged.head()

,Region,Province,District,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1278,Lima,Lima,Lima,-12.049141,-77.021217,1.0,Chinese Restaurant,Seafood Restaurant,Bakery,College Academic Building,Dim Sum Restaurant,Miscellaneous Shop,Spa,Furniture / Home Store,Market,Bookstore
1279,Lima,Lima,Ancón,-11.774270,-77.170060,1.0,Seafood Restaurant,Beach,Dessert Shop,Farm,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant
1280,Lima,Lima,Ate,-12.050000,-76.966670,1.0,Park,Fast Food Restaurant,Dessert Shop,Fried Chicken Joint,Soccer Field,Soccer Stadium,Department Store,Seafood Restaurant,Burger Joint,Chinese Restaurant
1283,Lima,Lima,Carabayllo,-11.853330,-77.037780,0.0,Latin American Restaurant,Peruvian Restaurant,Falafel Restaurant,Food,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
1286,Lima,Lima,Cieneguilla,-12.133330,-76.816670,0.0,Historic Site,Stables,Peruvian Restaurant,Yoga Studio,Falafel Restaurant,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


In [42]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_limaclus = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters_lima)
ys = [i + x + (i*x)**2 for i in range(kclusters_lima)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(lima_merged['Latitude'], lima_merged['Longitude'], lima_merged['District'], lima_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_limaclus)
       
map_limaclus

In [43]:
#CLUSTER 1 - Lima
lima_merged.loc[lima_merged['Cluster Labels'] == 0, lima_merged.columns[[2] + list(range(5, lima_merged.shape[1]))]]

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1283,Carabayllo,0.0,Latin American Restaurant,Peruvian Restaurant,Falafel Restaurant,Food,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
1286,Cieneguilla,0.0,Historic Site,Stables,Peruvian Restaurant,Yoga Studio,Falafel Restaurant,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
1300,Pachacámac,0.0,Peruvian Restaurant,Plaza,Italian Restaurant,Breakfast Spot,City,Farm,Food,Flower Shop,Flea Market,Fish & Chips Shop
1302,Puente Piedra,0.0,Peruvian Restaurant,Yoga Studio,Farm,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
1324,Supe,0.0,Plaza,History Museum,Peruvian Restaurant,Shopping Mall,Falafel Restaurant,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
1325,Supe puerto,0.0,Plaza,History Museum,Peruvian Restaurant,Shopping Mall,Falafel Restaurant,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
1339,Asia,0.0,City,Peruvian Restaurant,Yoga Studio,Food,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
1344,Imperial,0.0,Hotel,Peruvian Restaurant,Bus Station,Plaza,Falafel Restaurant,Food,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant
1348,Pacaran,0.0,Latin American Restaurant,Peruvian Restaurant,Falafel Restaurant,Food,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
1353,Zuñiga,0.0,Plaza,Peruvian Restaurant,Hotel,Historic Site,Falafel Restaurant,Food,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant


In [44]:
#CLUSTER 2 - Lima
lima_merged.loc[lima_merged['Cluster Labels'] == 1, lima_merged.columns[[2] + list(range(5, lima_merged.shape[1]))]]

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1278,Lima,1.0,Chinese Restaurant,Seafood Restaurant,Bakery,College Academic Building,Dim Sum Restaurant,Miscellaneous Shop,Spa,Furniture / Home Store,Market,Bookstore
1279,Ancón,1.0,Seafood Restaurant,Beach,Dessert Shop,Farm,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant
1280,Ate,1.0,Park,Fast Food Restaurant,Dessert Shop,Fried Chicken Joint,Soccer Field,Soccer Stadium,Department Store,Seafood Restaurant,Burger Joint,Chinese Restaurant
1287,Comas,1.0,Soccer Field,Snack Place,Fried Chicken Joint,Fast Food Restaurant,Seafood Restaurant,Yoga Studio,Farm,Flea Market,Fish & Chips Shop,Farmers Market
1288,El Agustino,1.0,Fried Chicken Joint,Plaza,Bus Station,Cemetery,Seafood Restaurant,Food & Drink Shop,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant
1289,Independencia,1.0,Theater,Pizza Place,Furniture / Home Store,Yoga Studio,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
1293,Lince,1.0,Seafood Restaurant,Peruvian Restaurant,Park,Bakery,Restaurant,Café,Italian Restaurant,BBQ Joint,Market,Latin American Restaurant
1295,Lurigancho,1.0,Restaurant,Fried Chicken Joint,Chinese Restaurant,Gym,Grocery Store,Fast Food Restaurant,Peruvian Restaurant,Spa,BBQ Joint,Seafood Restaurant
1296,Lurín,1.0,Plaza,Market,Shopping Mall,Asian Restaurant,BBQ Joint,Grocery Store,Juice Bar,Breakfast Spot,Latin American Restaurant,Flower Shop
1297,Magdalena del Mar,1.0,Café,Italian Restaurant,Coffee Shop,Fried Chicken Joint,Pizza Place,Seafood Restaurant,Peruvian Restaurant,Cupcake Shop,Ice Cream Shop,Restaurant


In [45]:
#CLUSTER 3 - Lima
lima_merged.loc[lima_merged['Cluster Labels'] == 2, lima_merged.columns[[2] + list(range(5, lima_merged.shape[1]))]]

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1340,Calango,2.0,Seafood Restaurant,Plaza,Yoga Studio,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
1342,Chilca,2.0,Plaza,Asian Restaurant,Beach,Dim Sum Restaurant,Diner,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop
1346,Mala,2.0,Plaza,Recreation Center,Restaurant,Exhibit,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
1347,Nuevo imperial,2.0,Plaza,Campground,Yoga Studio,Farm,Food,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
1377,Laraos,2.0,Mountain,Plaza,Farm,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
1383,San Damián,2.0,Plaza,Yoga Studio,Farm,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
1400,Caleta de Carquin,2.0,Plaza,Scenic Lookout,Seafood Restaurant,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
1433,Laraos,2.0,Mountain,Plaza,Farm,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


In [46]:
#CLUSTER 4 - Lima
lima_merged.loc[lima_merged['Cluster Labels'] == 3, lima_merged.columns[[2] + list(range(5, lima_merged.shape[1]))]]

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1369,Carampoma,3.0,Mountain,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
1384,San Juan de Iris,3.0,Mountain,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


In [47]:
#CLUSTER 5 - Lima
lima_merged.loc[lima_merged['Cluster Labels'] == 4, lima_merged.columns[[2] + list(range(5, lima_merged.shape[1]))]]

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1362,San Miguel de acos,4.0,Latin American Restaurant,Food Truck,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
1366,Matucana,4.0,Latin American Restaurant,Motel,Farm,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
